In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [5]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False, # note the difference!
    download=True,
    transform=ToTensor()
)

In [6]:
batch_size = 64

# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape of X [N, C, H, W]: ', X.shape)
    print('Shape of y: ', y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [7]:
class NeuralNetwork(nn.Module):
    # define layers of a neural network
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    # specify how data will pass through the network
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Using {} device'.format(device))

Using cuda device


In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    
    print(f'Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [14]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n---------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
---------------------------------
loss: 2.298352 [    0/60000]
loss: 2.288232 [ 6400/60000]
loss: 2.277707 [12800/60000]
loss: 2.278783 [19200/60000]
loss: 2.254952 [25600/60000]
loss: 2.250320 [32000/60000]
loss: 2.241770 [38400/60000]
loss: 2.224581 [44800/60000]
loss: 2.225362 [51200/60000]
loss: 2.232355 [57600/60000]
Test error: 
 Accuracy: 51.4%, Avg loss: 2.215144 

Epoch 2
---------------------------------
loss: 2.203909 [    0/60000]
loss: 2.197314 [ 6400/60000]
loss: 2.174895 [12800/60000]
loss: 2.206419 [19200/60000]
loss: 2.149417 [25600/60000]
loss: 2.142252 [32000/60000]
loss: 2.130370 [38400/60000]
loss: 2.094595 [44800/60000]
loss: 2.110382 [51200/60000]
loss: 2.125355 [57600/60000]
Test error: 
 Accuracy: 51.7%, Avg loss: 2.094574 

Epoch 3
---------------------------------
loss: 2.070322 [    0/60000]
loss: 2.055387 [ 6400/60000]
loss: 2.017206 [12800/60000]
loss: 2.091304 [19200/60000]
loss: 1.975077 [25600/60000]
loss: 1.973566 [32000/60000]
loss: 1.952967 [

In [17]:
save_model = False
load_model = False

if save_model is True:
    torch.save(model.state_dict(), 'model.pth')
    print('Saved PyTorch model state to model.pth')

if load_model is True:
    model = NeuralNetwork()
    model.load_state_dict(torch.load('model.pth'))
    
    classes = [
        'T-shirt/top',
        'Trousers',
        'Pullover',
        'Dress',
        'Coat',
        'Sandal',
        'Shirt',
        'Sneaker',
        'Bag',
        'Ankle boot'
    ]
    
    model.eval()
    X, y = test_data[0][0], test_data[0][1]
    with torch.no_grad():
        pred = model(X)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Saved PyTorch model state to model.pth
Predicted: "Ankle boot", Actual: "Ankle boot"
